# AMPSphere v.2022-03

This is a notebook meant to form the set of notebooks used to analyze the data in AMPSphere and write the manuscript:

__AMPSphere: Global survey of prokaryotic antimicrobial peptides shaping microbiomes__

### Check the c_AMP density of species through different environments

We generate sets of c_AMPs by species and sample. Then group them by environment and select those species happening in at least 2 habitats with at least 10 samples in each. The number of non-redundant c_AMPs attributed to a given species per sample in each sample is compared using Mann-Whitney U test and later the p-values are adjusted using Holm-Sidak.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import combinations, chain
from scipy.stats import spearmanr, mannwhitneyu
from statsmodels.stats.multitest import multipletests as mt

In [2]:
# load data
data = pd.read_table('../data_folder/complete_amps_associated_taxonomy.tsv.gz')
metadata = pd.read_table("../data_folder/metadata.tsv.xz")
metadata = metadata.rename({'sample_accession': 'sample'}, axis=1)

In [3]:
# filter species-specific genes
data = data.loc[data.level == 'species', ['sample', 'source']]
data = data.groupby(['sample', 'source']).agg('size')
data = data.reset_index()
data = data.rename({'source': 'name',
                    0: 'amp_genes'}, axis=1)

In [4]:
genenvo = pd.read_table('../data_folder/general_envo_names.tsv.xz')

metadata = metadata.merge(on=['microontology', 'host_scientific_name', 'host_tax_id'],
                          right=genenvo,
                          how='outer')

metadata = metadata[['sample', 'general_envo_name']]

In [5]:
# calculate densities
sps = pd.read_table('../data_folder/bps-per-sample-per-species.tsv.xz',
                    sep='\t',
                    header='infer')

data = data.merge(on=['sample', 'name'], right=sps)

data['amp_density'] = data['amp_genes'] * 1e6 / data['nbps']

# add metadata
data = data.merge(on='sample', right=metadata)
data.general_envo_name = data.general_envo_name.replace('human saliva', 'human mouth')

In [6]:
def tukeyfence(df):
    itdf = pd.DataFrame()
    for record in df.groupby(['name', 'general_envo_name']):
        record = record[1]
        if len(record) > 9:
            q1, q3 = record['amp_density'].quantile([0.25,0.75])
            iqr = q3-q1
            ul, ll = q3+(1.5*iqr), q1-(1.5*iqr)
            record = record[(record.amp_density <= ul) & (record.amp_density >= ll)]
            itdf = pd.concat([itdf, record])
    return itdf

In [7]:
# remove outliers
df = tukeyfence(data)
df['p'] = df.amp_density / 1e6
df['moe'] = np.sqrt(df.p*(1-df.p)/df.nbps) * 1.96 * 1e6
df.drop('p', axis=1, inplace=True)
df['moe_pct'] = df.moe * 100 / df.amp_density
df.to_csv('species_amp_density_per_sample.tsv.gz',
          sep='\t',
          header=True,
          index=None)
df

,sample,name,amp_genes,taxid,nbps,amp_density,general_envo_name,moe,moe_pct
901690,SAMN06264886,0-14-0-20-30-16 sp002779075,2,44747,1128853,1.771710,groundwater,2.455462,138.592806
924151,SAMN06266445,0-14-0-20-30-16 sp002779075,1,44747,767610,1.302745,groundwater,2.553378,195.999872
989431,SAMN06344006,0-14-0-20-30-16 sp002779075,1,44747,613652,1.629588,groundwater,3.193990,195.999840
1158011,SAMN07960327,0-14-0-20-30-16 sp002779075,2,44747,1089727,1.835322,groundwater,2.543624,138.592802
1158180,SAMN07960328,0-14-0-20-30-16 sp002779075,2,44747,1100407,1.817509,groundwater,2.518937,138.592803
...,...,...,...,...,...,...,...,...,...
1660322,TARA_142_MES_0.22-3_G,Zunongwangia profunda,1,1762,4485880,0.222922,marine,0.436926,195.999978
1660513,TARA_145_MES_0.22-3_G,Zunongwangia profunda,1,1762,4258030,0.234850,marine,0.460307,195.999977
1660989,TARA_148b_MES_0.22-3_G,Zunongwangia profunda,3,1762,3802238,0.789009,marine,0.892847,113.160608
1661071,TARA_149_MES_0.22-3_G,Zunongwangia profunda,1,1762,3801975,0.263021,marine,0.515521,195.999974


In [8]:
# select species present in >=10 samples per habitats
k = df.groupby(['name', 'general_envo_name']).agg('size')
k = k[k >= 10]

# select species present in more than 1 habitat
k = k.reset_index().groupby('name').apply(lambda x: x.general_envo_name.tolist())
k = k[k.apply(lambda x: len(x)) > 1]
k = k.reset_index()

### Supplementary Table S8

In [9]:
# test difference in the amp_densitys among samples of
# the same species through different environments
test = []
for _, s, env in k.itertuples():
    combs = combinations(env, 2)
    for i, j in combs:
        n1 = df[(df.name == s) & (df.general_envo_name == i)]
        n2 = df[(df.name == s) & (df.general_envo_name == j)]
        u, p = mannwhitneyu(n1['amp_density'], n2['amp_density'])
        test.append((s, i, len(n1),
                     n1.amp_density.mean(), n1.amp_density.std(),
                     j, len(n2), n2.amp_density.mean(),
                     n2.amp_density.std(), p))

# format results
test = pd.DataFrame(test, columns=['species',
                                   'env1',
                                   'env1_samples',
                                   'amp_density_avg_e1',
                                   'amp_density_std_e1',
                                   'env2',
                                   'env2_samples',
                                   'amp_density_avg_e2',
                                   'amp_density_std_e2',
                                   'p_value'])

test

# adjust p-values
_, test['p_adj'], _, _ = mt(test['p_value'], method='hs')

# export
test.to_csv('species_amp_density_crossenvironment.tsv.gz', sep='\t', header=True, index=None)
test

/home/celio/miniconda3/lib/python3.9/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


,species,env1,env1_samples,amp_density_avg_e1,amp_density_std_e1,env2,env2_samples,amp_density_avg_e2,amp_density_std_e2,p_value,p_adj
0,0-14-0-80-60-11 sp002779455,groundwater,27,1.065536,0.559337,marine,25,0.987186,0.569810,6.209545e-01,1.000000
1,0-14-0-80-60-11 sp002779455,groundwater,27,1.065536,0.559337,soil,155,1.207563,0.834207,7.908466e-01,1.000000
2,0-14-0-80-60-11 sp002779455,groundwater,27,1.065536,0.559337,water associated,45,0.810040,0.405034,6.273530e-02,1.000000
3,0-14-0-80-60-11 sp002779455,marine,25,0.987186,0.569810,soil,155,1.207563,0.834207,4.466062e-01,1.000000
4,0-14-0-80-60-11 sp002779455,marine,25,0.987186,0.569810,water associated,45,0.810040,0.405034,2.296780e-01,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
9927,Zag111 sp004558955,human gut,55,161.060656,90.532428,primate gut,15,21.175229,18.581715,8.012767e-09,0.000068
9928,Zag111 sp004558955,pig gut,145,3.867173,3.154102,primate gut,15,21.175229,18.581715,6.541202e-05,0.335970
9929,Zavarzinella formosa,plant associated,15,1.579412,1.080763,soil,123,1.602291,1.331643,7.843780e-01,1.000000
9930,Zoogloea ramigera,activated sludge,113,1.057880,0.688160,wastewater,167,1.132519,0.752228,4.592241e-01,1.000000


In [10]:
### Generating statistics

# select environments with at least 100 samples containing AMPs
k = data[['sample', 'general_envo_name']].drop_duplicates()
k = k.general_envo_name.value_counts()
k = k[k>=100].index

# select pairs containing both environments in the list
x = test[(test.env1.isin(k)) & (test.env2.isin(k))]

n1 = len(set(x.species))
print(f'It was tested {n1} species')

n2 = len(set(x.loc[test.p_adj < 0.05, 'species']))
print(f'''Only {round((n2*100/n1), 2)}% ({n2}) of them
presented significant differences in their AMP density
across different environments''')

savg = (x.amp_density_avg_e1.sum() + x.amp_density_avg_e2.sum()) / (2*len(x))

mmax, mmin = [max(x.amp_density_avg_e1.max(), x.amp_density_avg_e2.max()),
              min(x.amp_density_avg_e1.min(), x.amp_density_avg_e2.min())]

print(f'''In average, it was observed {savg:.2} c_AMP genes per assembled Mbp for each species
per sample, ranging between {mmax:.4}-{mmin:.2}.''')

It was tested 3930 species
Only 38.96% (1531) of them
presented significant differences in their AMP density
across different environments
In average, it was observed 2.3e+01 c_AMP genes per assembled Mbp for each species
per sample, ranging between 2.329e+03-0.13.
